## Installing Packages

In [268]:
# !pip install --pre pycaret
# !pip install scikit-learn-intelex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 26.0 MB 29.4 MB/s 
     |████████████████████████████████| 300.3 MB 17 kB/s 
     |████████████████████████████████| 4.0 MB 48.7 MB/s 


In [269]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Importing Packages

In [270]:
import pandas as pd
from pycaret.regression import *

## Data PreProcessing

In [271]:
data = pd.read_excel('DATASET.xlsx')
data.head()

,Unnamed: 0,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,Sec_1,1,324.0,354.5,17.0,0.0,3,117,2600,400,0.0,0.0,0.0,0,1.52
1,NaN,2,324.0,161.1,18.2,0.0,4,106,5950,1190,0.0,3.4,0.0,0,1.62
2,NaN,3,324.0,170.7,18.5,0.0,4,110,5950,1190,0.0,21.3,0.0,4,1.68
3,NaN,4,324.0,223.9,18.9,0.0,3,110,6150,1340,0.0,21.3,0.0,5,1.78
4,NaN,5,324.0,228.2,18.4,0.0,2,113,6340,1450,0.0,23.9,0.0,7,1.80


### Change column names


In [272]:
data.columns = data.columns.str.replace('Unnamed: 0', 'Sections')
data.columns

Index(['Sections', 'year', 'Para-1', 'Para-2', 'Para-3', 'Para-4', 'Para-5',
       'Para-6', 'Para-7', 'Para-8', 'Para-9', 'Para-10', 'Para-11', 'Para-12',
       'Para-13'],
      dtype='object')

### Populate "Sections" column

**Checking if every section contains data for 10 years**

In [273]:
data['year'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [274]:
missing_rows = []
i = 0
while i < len(data):
  section_name = data.iloc[i][0] if data.iloc[i][0] else i
  count = 0
  missing_entries=0

  while count < 10:
    if data.iloc[i+count][1] != count+1:
      missing_entries = missing_entries + 1
      missing_rows.append(i+count)
    count = count + 1
  i = i + 10 - missing_entries

**Since we can not remove a section altogether and we only have 1 missing row, lets duplicate 9th year record for the 10th one**




In [275]:
missing_rows

[919]

In [276]:
for i in missing_rows:
  new_idx = i - 0.5
  dup_rec_idx = i - 1
  data.loc[new_idx] = data.loc[dup_rec_idx]
  data.at[new_idx, 'year'] = i % 10 + 1
  data = data.sort_index().reset_index(drop=True)
data

,Sections,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,Sec_1,1,324.0,354.5,17.0,0.0,3,117,2600,400,0.0,0.0,0.0,0,1.520
1,NaN,2,324.0,161.1,18.2,0.0,4,106,5950,1190,0.0,3.4,0.0,0,1.620
2,NaN,3,324.0,170.7,18.5,0.0,4,110,5950,1190,0.0,21.3,0.0,4,1.680
3,NaN,4,324.0,223.9,18.9,0.0,3,110,6150,1340,0.0,21.3,0.0,5,1.780
4,NaN,5,324.0,228.2,18.4,0.0,2,113,6340,1450,0.0,23.9,0.0,7,1.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,NaN,6,454.0,931.8,13.6,125.0,13,116,4150,830,15.1,1.8,1.0,15,1.681
1006,NaN,7,454.0,703.0,14.5,17.0,13,118,4150,830,24.6,7.9,2.0,15,1.838
1007,NaN,8,454.0,1124.1,14.9,45.0,18,117,4200,840,119.0,15.7,2.0,16,1.862
1008,NaN,9,454.0,1462.3,13.2,62.0,27,117,4400,720,138.2,30.5,6.0,18,1.883


In [277]:
# Populating Sections column with values
j = 0
for i in range(int(len(data)/10)):
    data.iloc[j:j+10, 0] = data.iloc[j, 0]
    j = j + 10
data

,Sections,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
0,Sec_1,1,324.0,354.5,17.0,0.0,3,117,2600,400,0.0,0.0,0.0,0,1.520
1,Sec_1,2,324.0,161.1,18.2,0.0,4,106,5950,1190,0.0,3.4,0.0,0,1.620
2,Sec_1,3,324.0,170.7,18.5,0.0,4,110,5950,1190,0.0,21.3,0.0,4,1.680
3,Sec_1,4,324.0,223.9,18.9,0.0,3,110,6150,1340,0.0,21.3,0.0,5,1.780
4,Sec_1,5,324.0,228.2,18.4,0.0,2,113,6340,1450,0.0,23.9,0.0,7,1.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,Sec_101,6,454.0,931.8,13.6,125.0,13,116,4150,830,15.1,1.8,1.0,15,1.681
1006,Sec_101,7,454.0,703.0,14.5,17.0,13,118,4150,830,24.6,7.9,2.0,15,1.838
1007,Sec_101,8,454.0,1124.1,14.9,45.0,18,117,4200,840,119.0,15.7,2.0,16,1.862
1008,Sec_101,9,454.0,1462.3,13.2,62.0,27,117,4400,720,138.2,30.5,6.0,18,1.883


### Handle Nan Values


In [278]:
data.isnull().any()

Sections    False
year        False
Para-1      False
Para-2      False
Para-3      False
Para-4      False
Para-5      False
Para-6      False
Para-7      False
Para-8      False
Para-9      False
Para-10     False
Para-11     False
Para-12     False
Para-13      True
dtype: bool

In [279]:
data[data['Para-13'].isnull()]

,Sections,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
190,Sec_20,1,454.6,145.6,21.7,0.0,3,106,3650,730,0.0,0.0,0.0,0,NaN
260,Sec_27,1,563.9,336.6,21.3,0.0,4,106,6800,1600,0.0,0.0,0.0,0,NaN


In [280]:
data['Para-13'] = data['Para-13'].fillna(0)
data['Para-13'].isnull().any()

False

### Train - Test split

In [281]:
train_data = data[data.iloc[: , 1] != 10]
test_data = data[data.iloc[:, 1] == 10]

## Training

In [283]:
models = []
for i in range(9, 14):
  this_data = train_data.iloc[:, 0:10]
  this_data[f'Para-{i}'] = train_data[f'Para-{i}']
  s = setup(this_data, target=f'Para-{i}')
  best = compare_models(sort='RMSE', n_select=3)
  for model in best:
    models.append(tune_model(model))

,Description,Value
0,Session id,7853
1,Target,Para-9
2,Target type,Regression
3,Data shape,"(909, 11)"
4,Train data shape,"(636, 11)"
5,Test data shape,"(273, 11)"
6,Numeric features,9
7,Categorical features,1
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,12.0818,665.3203,25.1692,0.8193,1.1649,2.2484,0.1530
rf,Random Forest Regressor,11.1715,744.8760,26.0918,0.8264,0.9221,1.5227,0.1580
et,Extra Trees Regressor,11.4206,749.9041,26.1927,0.8261,0.9414,1.7252,0.2710
lightgbm,Light Gradient Boosting Machine,13.6743,923.4773,29.2674,0.7811,1.2228,2.3581,0.1600
ada,AdaBoost Regressor,29.5827,1505.3387,38.3556,0.5938,2.4870,9.7322,0.1320
dt,Decision Tree Regressor,15.4355,1727.7947,39.1767,0.5991,1.0307,1.7257,0.0480
en,Elastic Net,27.7414,1976.4060,43.1953,0.5276,2.2445,6.7848,0.0460
br,Bayesian Ridge,27.8816,1976.7640,43.2150,0.5269,2.2534,6.8696,0.0450
lasso,Lasso Regression,28.3341,1977.3773,43.2679,0.5247,2.2804,7.0358,0.1090
ridge,Ridge Regression,28.4863,1978.0846,43.2947,0.5237,2.2901,7.1442,0.0460


Processing:   0%|          | 0/79 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17.1544,1105.6456,33.2513,0.8929,1.4775,1.7313
1,15.9805,1183.5710,34.4031,0.7528,1.4802,3.5168
2,15.1171,720.1226,26.8351,0.8035,1.4262,2.0238
3,23.1476,1805.0815,42.4863,0.7654,1.6951,4.5530
4,19.6028,1431.7621,37.8386,0.7588,1.5687,3.9020
5,12.6317,600.1645,24.4983,0.6081,1.5261,9.8830
6,18.0627,909.6090,30.1597,0.6580,1.6299,5.6963
7,15.5262,686.3303,26.1979,0.8266,1.6911,4.3904
8,13.9391,484.7309,22.0166,0.8310,1.6073,3.1483


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1415,0.0761,0.2759,0.7840,0.1242,0.1129
1,0.2587,1.4414,1.2006,0.1356,0.2330,0.0960
2,0.1401,0.0443,0.2104,0.8591,0.0828,0.1107
3,0.1332,0.0363,0.1904,0.8954,0.0935,0.1601
4,0.1276,0.0413,0.2033,0.7988,0.0850,0.1141
5,0.1456,0.0724,0.2692,0.7461,0.0999,0.1178
6,0.1242,0.0611,0.2471,0.7135,0.0944,0.1099
7,0.1265,0.0368,0.1918,0.9040,0.0878,0.0996
8,0.1893,0.0925,0.3042,0.7297,0.1205,0.1457


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
# models_rmse_table contains total square deviation of different models for different "Para"
models_rmse_table = pd.DataFrame(range(len(models)), columns=['models']).set_index('models')

for i in range(10,14):
  this_data = train_data.iloc[:, 0:10]
  this_data[f'Para-{i}'] = train_data[f'Para-{i}']
  s = setup(this_data, target=f'Para-{i}')
  this_rmse = []
  for model in models:
    prediction = predict_model(model, test_data.iloc[:, 0:10])['prediction_label']
    this_rmse.append(sum((prediction - test_data[f'Para-{i}'].reset_index(drop=True))**2))
  models_rmse_table[f'Para-{i}'] = this_rmse
models_rmse_table

In [297]:
final_rmse = []
for i in range(len(models_rmse_table)):
  final_rmse.append((sum(models_rmse_table.iloc[i, :])/(5*len(test_data)))**0.5)
final_rmse

[47.34033208766921,
 41.575826924482826,
 46.78586265223186,
 37.37720719042681,
 38.07211707902503,
 39.54677483196249,
 52.05672842033515,
 52.888743458301406,
 57.89908013801939,
 87.95396232820195,
 88.29155131471478,
 88.3731568951671,
 96.40139430239479,
 57.18324587866645,
 57.18324587866625]

In [298]:
min_idx = final_rmse.index(min(final_rmse))
print(models[min_idx], 'with RMSE = ', final_rmse[min_idx])

ExtraTreesRegressor(n_jobs=-1, random_state=4637) with RMSE =  37.37720719042681
